In [ ]:
import pandas as pd

stores = pd.read_csv("datos/stores.csv")
providers = pd.read_csv("datos/providers1.csv")
visits = pd.read_csv("datos/visits_1.csv")
answer = pd.read_csv("datos/answer.csv")
types = pd.read_csv("datos/types.csv")
# Renombrar columnas con sufijo por tabla
stores = stores.add_suffix("_store")
providers = providers.add_suffix("_provider")
visits = visits.add_suffix("_visit")
answer = answer.add_suffix("_answer")
type = types.add_suffix("_type")
# Unir tablas
df_filtrado = visits.merge(
    answer, left_on="id_visit", right_on="visit_id_answer", how="left"
)
df_filtrado = df_filtrado.merge(
    stores, left_on="store_id_visit", right_on="id_store", how="left"
)
df_filtrado = df_filtrado.merge(
    providers, left_on="user_id_visit", right_on="id_user_provider", how="left"
)
df_filtrado = df_filtrado.merge(
    type, left_on="type_id_visit", right_on="id_type", how="left"
)
df_filtrado.head()
df_filtrado.columns

Index(['created_at_visit', 'Cruce_visit', 'DiaMes_visit', 'ended_at_visit',
       'fecha_visit', 'Fecha_Menos6h_visit', 'FechaDia_visit',
       'FechaHora_Menos6h_visit', 'id_visit', 'Incidencia válida_visit',
       'OrdenDia_visit', 'Periodo_visit', 'Periodo.1_visit', 'DiaMes.1_visit',
       'status_visit', 'store_id_visit', 'Tamaño_visit', 'type_id_visit',
       'user_id_visit', 'answer_answer', 'Año_answer', 'Trimestre_answer',
       'Mes_answer', 'Día_answer', 'id_answer', 'Incidencia_answer',
       'question_id_answer', 'sectionid_answer', 'TamañoAsignado_answer',
       'type_id_answer', 'Año.1_answer', 'Trimestre.1_answer', 'Mes.1_answer',
       'Día.1_answer', 'visit_id_answer', 'id_store', 'store_zone_store',
       'store_name_store', 'store_mr_store', 'store_mr.1_store',
       'store_operative_store', 'store_sap_store', 'store_region_store',
       'code_provider', 'email_provider', 'id_proveedor_provider',
       'name_provider', 'status_provider', 'full_name_user_

In [ ]:
#Eliminar duplicados de id_vist y traer la info

tabla = df_filtrado.drop_duplicates(subset=["id_visit"])[[
        "id_visit",
        "store_sap_store",
        "store_name_store",
        "store_zone_store",
        "store_region_store",
        "name_type",
        "FechaHora_Menos6h_visit",
        "status_visit",
        "name_provider"

]]
tabla.head()


,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA
5,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA
10,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
12,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
17,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA


In [ ]:
# Calculo tiene incidencia
incidencias = df_filtrado[df_filtrado["sectionid_answer"].isin([2,5])] #Se utiliza isin para filtrar los valores de la encuesta que tengan ese valor
incidencias = incidencias.groupby("id_visit").size().reset_index(name="count_incidencias") #cuenta la cantidad de incidencias que hay en cada visita y lo vuelve dataframe
incidencias["TieneIncidencia"] = incidencias["count_incidencias"].apply(lambda x: "Sí" if x > 0 else "No")

incidencias.head()




,id_visit,count_incidencias,TieneIncidencia
0,2555,2,Sí
1,2562,2,Sí
2,2564,2,Sí
3,2572,3,Sí
4,2612,2,Sí


In [95]:
tabla = tabla.merge(incidencias[["id_visit", "TieneIncidencia"]], 
                    on="id_visit", 
                    how="left")

tabla[tabla["id_visit"] == 10485]

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia
1509,10485,120155808,MISC NONNY,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-25 12:07:07,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN


In [ ]:
# Calculo de Motivo
motivo = df_filtrado[df_filtrado["question_id_answer"].isin([8,5])]
motivo = motivo.groupby("id_visit")["answer_answer"].first().reset_index(name="Respuesta_motivo")

motivo.head()


,id_visit,Respuesta_motivo
0,2555,Cliente Cerrado Lona
1,2562,Cliente Cerrado Lona
2,2564,Cliente no permitió cambio
3,2572,Cliente no permitió cambio
4,2612,Cliente no tiene Lona


In [96]:
tabla = tabla.merge(motivo, 
                    on="id_visit", 
                    how="left")

tabla.head()

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia,Respuesta_motivo
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA,NaN,NaN
1,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA,NaN,NaN
2,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,Sí,Cliente Cerrado Lona
3,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN
4,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN


In [ ]:
# ============= Tamaño Banner / QR
tamaño = df_filtrado[df_filtrado["question_id_answer"].isin([4,14])]
tamaño = tamaño.groupby("id_visit")["answer_answer"].first().reset_index(name="tamaño")

tamaño.head()

,id_visit,tamaño
0,48,Grande
1,55,Grande
2,2556,Básico
3,2560,Grande
4,2561,Básico


In [99]:
tabla = tabla.merge(tamaño,
                    on="id_visit",
                    how = "left")

tabla.head()

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia,Respuesta_motivo,tamaño
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande
1,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande
2,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,Sí,Cliente Cerrado Lona,NaN
3,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Básico
4,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Grande


In [ ]:
# ========== Comentario
comentario = df_filtrado[df_filtrado["question_id_answer"].isin([7, 10])]
comentario = comentario.groupby("id_visit")["answer_answer"].first().reset_index(name = "comentario")

tabla = tabla.merge(comentario,
                    on="id_visit",
                    how = "left")

tabla.head()

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia,Respuesta_motivo,tamaño,comentario
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande,NaN
1,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande,NaN
2,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,Sí,Cliente Cerrado Lona,NaN,NaN
3,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Básico,NaN
4,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Grande,NaN


In [ ]:
tabla_final = tabla[[
    "store_sap_store",
    "store_name_store",
    "store_zone_store",
    "store_region_store",
    "name_type",
    "FechaHora_Menos6h_visit",
    "status_visit",
    "TieneIncidencia",
    "Respuesta_motivo",
    "name_provider",
    "tamaño",
    "comentario"
]]

In [104]:
# --- Seleccionar y renombrar columnas finales ---
tabla_final.rename(columns={
    "store_sap_store": "SAP",
    "store_name_store": "PDV",
    "store_zone_store": "ZONA",
    "store_region_store": "REGION",
    "name_type": "CATEGORÍA",
    "FechaHora_Menos6h_visit": "FECHA VISITA",
    "status_visit": "ESTADO",
    "TieneIncidencia": "TIENE INCIDENCIA",
    "Respuesta_motivo": "MOTIVO",
    "name_provider": "PROVEEDOR",
    "tamaño": "TAMAÑO",
    "comentario": "COMENTARIO"
})

,SAP,PDV,ZONA,REGION,CATEGORÍA,FECHA VISITA,ESTADO,TIENE INCIDENCIA,MOTIVO,PROVEEDOR,TAMAÑO,COMENTARIO
0,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,NaN,NaN,CORPORATIVO MDA,Grande,NaN
1,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,NaN,NaN,CORPORATIVO MDA,Grande,NaN
2,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,Sí,Cliente Cerrado Lona,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN
3,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Básico,NaN
4,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1641,101002602,MISC ROSITA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 10:30:06,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
1642,120290750,MISC TU Y YO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-29 10:04:14,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
1643,120301264,TIENDA BEATRIZ,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-31 15:03:23,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
1644,120355365,TIENDA NANCY,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-29 19:36:44,Finalizado,Sí,Cliente no permitió cambio,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,No sabe de promoción


In [105]:
tabla["status_visit"].value_counts()


status_visit
Finalizado     1644
En progreso       1
Pendiente         1
Name: count, dtype: int64

In [106]:
tabla["status_visit"].unique()


array(['Finalizado', 'En progreso', 'Pendiente'], dtype=object)

In [107]:
df_inc = (
    tabla
    .groupby("status_visit")["id_visit"]
    .nunique()
    .reset_index()
    .rename(columns={"id_visit": "total"})
)

print(df_inc)

  status_visit  total
0  En progreso      1
1   Finalizado   1644
2    Pendiente      1
